# Enron Case-Person of Interest

In [ ]:
#!/usr/bin/python
## Author is Rajdeep
import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
##  tester.py is modified becuase of renaming and deprecation of cross_validation sub-module to model_selection
from tester import dump_classifier_and_data
##  tester.py is modified becuase of renaming and deprecation of cross_validation sub-module to model_selection

In [2]:
### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".
features_list=['poi','salary']

In [3]:
### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)

In [4]:
### Data exploration
### Task 2: Remove outliers
### Task 3: Create new feature(s)
### Store to my_dataset for easy export below.
my_dataset = data_dict

In [5]:
#Number of suspects
name_data_point=data_dict.keys()

In [6]:
print 'Total number of datapoints are {}'.format(len(name_data_point))

Total number of datapoints are 146


In [7]:
#Number of features
total_features=len (data_dict[name_data_point[0]])
print 'Total number of features are {}'.format(total_features)


Total number of features are 21


In [48]:
features_temp=data_dict[name_data_point[0]].keys()

In [51]:
#For convinience of data cleaning (Removing NaN) using pandas (Ref: Data visulization)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [52]:
unwanted_features=["poi","email_address", "restricted_stock_deferred","loan_advances","director_fees", "other","deferral_payments","deferred_income"]

features_temp=[ele for ele in features_temp if ele not in unwanted_features]
# to make the first element in the list "poi"
features_temp=["poi"]+features_temp
feature_data= featureFormat(data_dict,features_temp,remove_NaN=False)


In [53]:
temp_df=pd.DataFrame(data=feature_data,columns=features_temp, index=name_data_point)
print "With NaN"
print temp_df.info()

With NaN
<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, METTS MARK to GLISAN JR BEN F
Data columns (total 14 columns):
poi                        146 non-null float64
salary                     95 non-null float64
to_messages                86 non-null float64
total_payments             125 non-null float64
exercised_stock_options    102 non-null float64
bonus                      82 non-null float64
restricted_stock           110 non-null float64
shared_receipt_with_poi    86 non-null float64
total_stock_value          126 non-null float64
expenses                   95 non-null float64
from_messages              86 non-null float64
from_this_person_to_poi    86 non-null float64
long_term_incentive        66 non-null float64
from_poi_to_this_person    86 non-null float64
dtypes: float64(14)
memory usage: 17.1+ KB
None


In [54]:
#Replacing 'NaN' with 0
temp_df=temp_df.fillna(0)
print "Removing NaN and replacing with 0"
print temp_df.info()

Removing NaN and replacing with 0
<class 'pandas.core.frame.DataFrame'>
Index: 146 entries, METTS MARK to GLISAN JR BEN F
Data columns (total 14 columns):
poi                        146 non-null float64
salary                     146 non-null float64
to_messages                146 non-null float64
total_payments             146 non-null float64
exercised_stock_options    146 non-null float64
bonus                      146 non-null float64
restricted_stock           146 non-null float64
shared_receipt_with_poi    146 non-null float64
total_stock_value          146 non-null float64
expenses                   146 non-null float64
from_messages              146 non-null float64
from_this_person_to_poi    146 non-null float64
long_term_incentive        146 non-null float64
from_poi_to_this_person    146 non-null float64
dtypes: float64(14)
memory usage: 17.1+ KB
None


## Outlier detection and data cleaning 

In [55]:
# Determining the outliers for financial datas
financial_features=["salary", "total_payments","exercised_stock_options","bonus","restricted_stock", "total_stock_value", "expenses"]
temp_df[financial_features].describe()

,salary,total_payments,exercised_stock_options,bonus,restricted_stock,total_stock_value,expenses
count,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02,1.460000e+02
mean,3.658114e+05,4.350622e+06,4.182736e+06,1.333474e+06,1.749257e+06,5.846018e+06,7.074827e+04
std,2.203575e+06,2.693448e+07,2.607040e+07,8.094029e+06,1.089995e+07,3.624681e+07,4.327163e+05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.604490e+06,-4.409300e+04,0.000000e+00
25%,0.000000e+00,9.394475e+04,0.000000e+00,0.000000e+00,8.115000e+03,2.288695e+05,0.000000e+00
50%,2.105960e+05,9.413595e+05,6.082935e+05,3.000000e+05,3.605280e+05,9.659550e+05,2.018200e+04
75%,2.708505e+05,1.968287e+06,1.714221e+06,8.000000e+05,8.145280e+05,2.319991e+06,5.374075e+04
max,2.670423e+07,3.098866e+08,3.117640e+08,9.734362e+07,1.303223e+08,4.345095e+08,5.235198e+06


In [56]:
temp_df[temp_df["restricted_stock"]<0]
## Based on my understanding restricted stocks or any stocks cannot be negative. And this is only one with negative restricted_stocks
temp_df=temp_df.drop(index="BHATNAGAR SANJAY",axis=0)

In [57]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145 entries, METTS MARK to GLISAN JR BEN F
Data columns (total 14 columns):
poi                        145 non-null float64
salary                     145 non-null float64
to_messages                145 non-null float64
total_payments             145 non-null float64
exercised_stock_options    145 non-null float64
bonus                      145 non-null float64
restricted_stock           145 non-null float64
shared_receipt_with_poi    145 non-null float64
total_stock_value          145 non-null float64
expenses                   145 non-null float64
from_messages              145 non-null float64
from_this_person_to_poi    145 non-null float64
long_term_incentive        145 non-null float64
from_poi_to_this_person    145 non-null float64
dtypes: float64(14)
memory usage: 17.0+ KB


In [58]:
temp_df[temp_df["salary"]==temp_df["salary"].max()]
# TOTAL seems odd, and it was pointed out in the session that this could be a human error hence this a outlier too.


,poi,salary,to_messages,total_payments,exercised_stock_options,bonus,restricted_stock,shared_receipt_with_poi,total_stock_value,expenses,from_messages,from_this_person_to_poi,long_term_incentive,from_poi_to_this_person
TOTAL,0.0,26704229.0,0.0,309886585.0,311764000.0,97343619.0,130322299.0,0.0,434509511.0,5235198.0,0.0,0.0,48521928.0,0.0


In [59]:
temp_df=temp_df.drop(index="TOTAL",axis=0)

In [60]:
temp_df=temp_df.drop(index=temp_df[temp_df["total_stock_value"]<0].index,axis=0)

In [61]:
temp_df[financial_features].loc[(temp_df[financial_features]==0).all(axis=1)].index
# found this two outliers with all finantial features as zero, hence it can be dropped

Index([u'POWERS WILLIAM', u'LOCKHART EUGENE E'], dtype='object')

In [62]:
temp_df=temp_df.drop(temp_df[financial_features].loc[(temp_df[financial_features]==0).all(axis=1)].index,axis=0)

In [64]:
#converting back to dict
data_dict=temp_df.to_dict('index')

In [66]:
# Reference module from feature selection: Used for creating new features "fraction_to_poi" and "fraction_from_poi"
def computeFraction( poi_messages, all_messages ):

    if poi_messages!=0 and all_messages!=0:
        fraction = float(poi_messages)/float(all_messages)
        return fraction
    else:
        fraction = 0.0
    



        return fraction

In [67]:
# Used for creating new features "total_profit"
def total_profit(bonus, salary,long_term_incentive):
    profit=float(bonus)+float(salary)+float(long_term_incentive)
    
    return profit
    

In [68]:
submit_dict = {}
for name in data_dict:

    data_point = data_dict[name]

    
    from_poi_to_this_person = data_point["from_poi_to_this_person"]
    to_messages = data_point["to_messages"]
    fraction_from_poi = computeFraction( from_poi_to_this_person, to_messages )
    
    data_point["fraction_from_poi"] = fraction_from_poi
    
    bonus=data_point["bonus"]
    salary=data_point["salary"]
    long_term_incentive=data_point["long_term_incentive"]


    from_this_person_to_poi = data_point["from_this_person_to_poi"]
    from_messages = data_point["from_messages"]
    fraction_to_poi = computeFraction( from_this_person_to_poi, from_messages )
    profit=total_profit(bonus, salary,long_term_incentive)
    submit_dict[name]={"from_poi_to_this_person":fraction_from_poi,
                       "from_this_person_to_poi":fraction_to_poi,
                      "total_profit":total_profit}
    data_point["fraction_to_poi"] = fraction_to_poi
    data_dict[name]["fraction_to_poi"]=fraction_to_poi
    data_dict[name]["fraction_from_poi"]= fraction_from_poi
    data_dict[name]["total_profit"]=profit

In [69]:
#added two additional features in data_dict
total_features=len (data_dict[name_data_point[0]])
print 'After cleaning Total number of features are {}'.format(total_features)
data_dict[name_data_point[0]]

Total number of features are 17


{'bonus': 600000.0,
 'exercised_stock_options': 0.0,
 'expenses': 94299.0,
 'fraction_from_poi': 0.04708798017348203,
 'fraction_to_poi': 0.034482758620689655,
 'from_messages': 29.0,
 'from_poi_to_this_person': 38.0,
 'from_this_person_to_poi': 1.0,
 'long_term_incentive': 0.0,
 'poi': 0.0,
 'restricted_stock': 585062.0,
 'salary': 365788.0,
 'shared_receipt_with_poi': 702.0,
 'to_messages': 807.0,
 'total_payments': 1061827.0,
 'total_profit': 965788.0,
 'total_stock_value': 585062.0}

In [75]:
print 'After cleaning number of data points {}'.format(len(data_dict))

After cleaning number of data points 141


In [76]:
features_list=features_temp
data = featureFormat(my_dataset, features_list, sort_keys = True)
labels, features = targetFeatureSplit(data)

## Feature Scaling

In [77]:
from sklearn.model_selection import train_test_split

In [78]:
features_train, features_test, labels_train, labels_test =train_test_split(features, labels, test_size=0.3, random_state=42)

In [85]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [86]:
from time import time

In [87]:
t0=time()
clf.fit(features_train,labels_train)
print "Time to train {}".format(round(time()-t0,3)), 's'

Time to train 0.048 s


In [89]:
# Accuracy of Gaussian classifier
print "Accuracy for Naive Gaussian", clf.score(features_test,labels_test)

Accuracy for Naive Gaussian 0.13636363636363635


In [141]:
# Using max_min scaler as I am using SVC which is effected by scale of dimensions
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif

scaler = MinMaxScaler()
clf=SVC()
skb=SelectKBest(f_classif)
pipe = Pipeline([("SKB", skb), ('scaler', scaler), ('SVC', clf)])


In [146]:
from sklearn.model_selection import GridSearchCV
param_grid = {"SKB__k": [3,4,5,6,7,10,13],'SVC__C': [1, 10, 100, 1000], 'SVC__gamma': [0.001, 0.0001], 'SVC__kernel': ['linear','rbf']}
clf_grid=GridSearchCV(pipe,param_grid)
clf.get_params().keys()

['kernel',
 'C',
 'verbose',
 'probability',
 'degree',
 'shrinking',
 'max_iter',
 'decision_function_shape',
 'random_state',
 'tol',
 'cache_size',
 'coef0',
 'gamma',
 'class_weight']

In [147]:
t0=time()
clf_grid.fit(features, labels)
print "Time to train SVC {}".format(round(time()-t0,3)), 's'

Time to train SVC 4.279 s


C:\Users\RATAN\Anaconda2\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [148]:
clf_grid.best_estimator_

Pipeline(memory=None,
     steps=[('SKB', SelectKBest(k=3, score_func=<function f_classif at 0x000000000D5CC0B8>)), ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('SVC', SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

In [118]:
clf=clf_grid.best_estimator_
clf.fit(features_train,labels_train)
prediction=clf.predict(features_test)

In [159]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print "Accuracy of SVC classifer is  : ",accuracy_score(labels_test, prediction)
print "Precision of SVC classifer is : ",precision_score(labels_test,prediction,average='weighted')
print "Recall of SVC classifer is    : ",recall_score(labels_test,prediction,average='weighted')
print "f1-score of SVC classifer is  : ",f1_score(labels_test,prediction,average='weighted')

Accuracy of SVC classifer is  :  0.8636363636363636
Precision of SVC classifer is :  0.7458677685950413
Recall of SVC classifer is    :  0.8636363636363636
f1-score of SVC classifer is  :  0.8004434589800443
